In [ ]:
import azureml
from azureml.core import Workspace
workspaces = Workspace.list("3165a1c1-fd45-4c8d-938e-0058c823f960")
ws = workspaces["aml-playground"]
playground_ws = ws[0]
playground_ws

In [ ]:
import mlflow

## Construct AzureML MLFLOW TRACKING URI
def get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace):
    return "azureml://{}.api.azureml.ms/mlflow/v1.0/subscriptions/{}/resourceGroups/{}/providers/Microsoft.MachineLearningServices/workspaces/{}".format(region, subscription_id, resource_group, workspace)

region='northeurope' ## example: westus
subscription_id = '3165a1c1-fd45-4c8d-938e-0058c823f960' ## example: 11111111-1111-1111-1111-111111111111
resource_group = 'aml-playground' ## example: myresourcegroup
workspace = 'aml-playground' ## example: myworkspacename

MLFLOW_TRACKING_URI = get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace)

## Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## Make sure the MLflow URI looks something like this: 
## azureml://<REGION>.api.azureml.ms/mlflow/v1.0/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.MachineLearningServices/workspaces/<AML_WORKSPACE_NAME>

print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
cluster_name = "preproject-cluster"
# instance_name = "cpuinstance1"
# cluster_name = instance_name


try:
    compute_target = ComputeTarget(workspace=playground_ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=1)

    # Create the cluster.
    compute_target = ComputeTarget.create(playground_ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current AmlCompute.
print(compute_target.get_status().serialize())

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.environment import Environment as AmlEnvironment

auv_docker_env = AmlEnvironment.from_docker_image("preproject", "krisbrud.azurecr.io/preproject")
# experiment_name = ""


src = ScriptRunConfig(source_directory='../gym-auv',
                      script='train3d.py',
                      arguments=["--config", "surge_assisted"],
                      compute_target=compute_target,
                      environment=auv_docker_env)